In [460]:
import pandas as pd
import csv

file = pd.read_csv('./Input/data/no_mask/no_mask.csv')
file.head()

,db,full_path,age,gender
0,UTKface,part1/41_0_2_20170105164126307.jpg,41,1
1,UTKface,part1/80_1_0_20170110122217473.jpg,80,0
2,UTKface,part1/56_0_0_20170111202842202.jpg,56,1
3,UTKface,part1/82_1_0_20170110141329969.jpg,82,0
4,UTKface,part1/16_1_4_20170102234841875.jpg,16,0


In [454]:
file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24104 entries, 0 to 24103
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   db         24104 non-null  object
 1   full_path  24104 non-null  object
 2   age        24104 non-null  int64 
 3   gender     24104 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 753.4+ KB


In [469]:
subfile = file

In [470]:
sub_file['state'] = ['normal' for i in sub_file.index]

In [471]:
sub_file = sub_file.loc[:,['gender','age','state']]

In [484]:
over_60 = sub_file['age'] == '(60, 100)' 
is_male = sub_file['gender'] == 'm' 
is_female = sub_file['gender'] == 'f'
sub_file = sub_file[over_60 & (is_male | is_female)]

In [485]:
sub_file.index

Int64Index([], dtype='int64')

In [486]:
sub_file['state'] = ['normal' for i in sub_file.index]

In [487]:
sub_file.sample(10)

ValueError: a must be greater than 0 unless no samples are taken

In [488]:
sub_file.count()

gender    0
age       0
state     0
dtype: int64

In [ ]:
sub_file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 928 entries, 744 to 22034
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   gender  928 non-null    object
 1   age     928 non-null    object
 2   state   928 non-null    object
 3   path    928 non-null    object
dtypes: object(4)
memory usage: 36.2+ KB


In [ ]:
sub_file.describe()

,gender,age,state,path
count,928,928,928,928
unique,2,1,1,1
top,f,"(60, 100)",normal,./Input/data/no_mask/images/114841417@N06/
freq,466,928,928,928


In [ ]:
train_df = sub_file

In [ ]:
def labeling(df, folder_path):
    temp = pd.DataFrame()
    
    path_list = []
    full_path_list = [] 
    state_list = []
    for path in df["path"]:
        #사람 폴더에서 이미지 경로 받기.
        imgs = glob.glob(folder_path + "/" + path + "/*")
        
        path_list.extend([path]*7)
        full_path_list.extend([full_path for full_path in imgs])
        state_list.extend([full_path.split("/")[-1].split(".")[0] for full_path in imgs])
  
    temp["path"] = path_list
    temp["img_path"] = full_path_list
    temp["state"] = state_list
    return temp

# 기존 df와 생성된 df 합치기
def merge_df(df, mask_state_df):
    return_df = pd.merge(df, mask_state_df, how= "outer", on = "path")
    # print(return_df)
    return_df.loc[return_df.query("state == 'incorrect_mask'").index, "state"] = "incorrect"

    # 라벨링
    return_df["label"] = 0
    return_df.loc[return_df.query("state == 'incorrect'").index, "label"] += 6
    return_df.loc[return_df.query("state == 'normal'").index, "label"] += 12

    return_df.loc[return_df.query("gender == 'female'").index, "label"] += 3

    return_df.loc[return_df.query("30 <= age < 60 ").index, "label"] += 1
    return_df.loc[return_df.query("age == 60 ").index, "label"] += 2


    # valid_df = return_df.iloc[200:]
    # return_df = return_df.iloc[:200]

    return_df["c_gender"] = 0
    return_df.loc[return_df.query("gender == 'male'").index, "c_gender"] = 1

    return_df["c_state"] = 0
    return_df.loc[return_df.query("state == 'incorrect'").index, "c_state"] = 1
    return_df.loc[return_df.query("state == 'normal'").index, "c_state"] = 2

    return_df["c_age"] = 0
    return_df.loc[return_df.query("30 <= age < 60 ").index, "c_age"] = 1
    return_df.loc[return_df.query("age == 60 ").index, "c_age"] = 2
    
    return_df["c_age_deep"] = 0
    return_df.loc[return_df.query("20 <= age < 25 ").index, "c_age_deep"] = 1
    return_df.loc[return_df.query("25 <= age < 30 ").index, "c_age_deep"] = 2
    return_df.loc[return_df.query("30 <= age < 35 ").index, "c_age_deep"] = 3
    return_df.loc[return_df.query("35 <= age < 40 ").index, "c_age_deep"] = 4
    return_df.loc[return_df.query("40 <= age < 45 ").index, "c_age_deep"] = 5
    return_df.loc[return_df.query("45 <= age < 50 ").index, "c_age_deep"] = 6
    return_df.loc[return_df.query("50 <= age < 55 ").index, "c_age_deep"] = 7
    return_df.loc[return_df.query("55 <= age < 60 ").index, "c_age_deep"] = 8
    return_df.loc[return_df.query("age == 60 ").index, "c_age_deep"] = 9

    
    
    label_dict = {0: "마스크쓴 남성 청년",
              1: "마스크쓴 남성 중년",
              2: "마스크쓴 남성 노년",
              3: "마스크쓴 여성 청년",
              4: "마스크쓴 여성 중년",
              5: "마스크쓴 여성 노년",
              
              6: "잘못쓴 남성 청년",
              7: "잘못쓴 남성 중년",
              8: "잘못쓴 남성 노년",
              9: "잘못쓴 여성 청년",
              10: "잘못쓴 여성 중년",
              11: "잘못쓴 여성 노년",
              
              12: "안쓴 남성 청년",
              13: "안쓴 남성 중년",
              14: "안쓴 남성 노년",
              15: "안쓴 여성 청년",
              16: "안쓴 여성 중년",
              17: "안쓴 여성 노년",
    }
    return_df["label_check"] = return_df["label"].replace(label_dict)
    return return_df


mask_state_train_df = labeling(train_df, "Input/data/no_mask/images")  
train_df = merge_df(train_df, mask_state_train_df)

train_df.sample(20)[["gender","age","state","label_check"]]

KeyError: 'path'